# What happens:

- sql is almost done
- all with foor loop
- add the coloring in the header and total

## Import 

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import datetime
import win32com.client as win32
import calendar
from openpyxl import Workbook, load_workbook
from IPython.lib.display import Audio
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.cell import Cell
from openpyxl.styles import Color, Font, PatternFill

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [2]:
directory = "C:/Users/WZHARBC/Documents/Python Monthly Tasks/Exports/ExAnte"
os.chdir(directory)

rd = str(input("Please enter the RD (YYYYMMDD): "))

ws = "0002"

Please enter the RD (YYYYMMDD): 20220930


### Make connection with Oracle DB

In [21]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                 reporting_date = rd, partition = ws))

### Open excel file

In [52]:
# determining the name of the file
file_name = 'ExAnte Notification ' + calendar.month_name[int(rd[4:6])] + ' ' + rd[0:4] + '.xlsx'

file = load_workbook("ExAnte Template.xlsx")

sqls = file['sql and workspace']


## Select SQL 1

In [53]:
sql1 = file['sql 1']

d1 = pd.read_sql_query("""select 
case when table_name = 'FUND' and contract_type ='IS2700'   then' Investment in FUND' else rating_model_new  end as rating_model_new_mapped
,rating_model_new,
sum (ead) ead ,sum (rwa) rwa
from rzb_cdr 
where record_source='T_CDR' and method_new = 'FOU' 
and NOT (table_name='EQUITY' and sub_method_new='STD')---to remove the grandfathering equities
group by case when table_name = 'FUND' and contract_type ='IS2700'   then' Investment in FUND' else rating_model_new  end, rating_model_new""", connection_RAY)

### Input the values and add the colors

In [54]:
# add headers
for idx_c, header in enumerate(d1.columns.values):
    sql1.cell(row=1, column=idx_c+1).value = header
    sql1.cell(row=1, column=idx_c+1).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

# Iterate all rows using DataFrame.itertuples()
index = 0
for idx, row in enumerate(d1.itertuples(index = True)):
    for idx_c, header in enumerate(d1.columns.values):
        sql1.cell(row=idx+2, column=idx_c+1).value = getattr(row, header)
        index = idx

# Add the total        
sql1.cell(row=index+3, column=2).value = 'Total:'
sql1.cell(row=index+3, column=3).value = '=SUM(C2:C'+str(index+2)+')'
sql1_total = str(index+3)
sql1.cell(row=index+3, column=4).value = '=SUM(D2:D'+str(index+2)+')'

for col in range(2, 5, 1):
    sql1.cell(row=index+3, column=col).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')


### Select SQL 2

In [55]:
sql2 = file['sql 2']

d2 = pd.read_sql_query("""select
case when table_name = 'FUND' and contract_type ='IS2700' then 'Investment in FUND' else  counterparty_new_type end as counterparty_new_type_mapped,
counterparty_new_type,counterparty_new_type_desc,sum (ead) ead, sum (rwa) rwa from rzb_cdr
where record_source='T_CDR' and method_new = 'FOU'
and NOT (table_name='EQUITY' and sub_method_new='STD')---to remove the grandfathering equities
group by case when table_name = 'FUND' and contract_type ='IS2700' then 'Investment in FUND' else  counterparty_new_type end,counterparty_new_type,counterparty_new_type_desc
""", connection_RAY)

### Input the values and add the colors

In [56]:
# add headers
for idx_c, header in enumerate(d2.columns.values):
    sql2.cell(row=1, column=idx_c+1).value = header
    sql2.cell(row=1, column=idx_c+1).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

# Iterate all rows using DataFrame.itertuples()
index = 0
for idx, row in enumerate(d2.itertuples(index = True)):
    for idx_c, header in enumerate(d2.columns.values):
        sql2.cell(row=idx+2, column=idx_c+1).value = getattr(row, header)
        index = idx

# Add the total        
sql2.cell(row=index+3, column=3).value = 'Total:'
sql2.cell(row=index+3, column=4).value = '=SUM(D2:D'+str(index+2)+')'
sql2.cell(row=index+3, column=5).value = '=SUM(E2:E'+str(index+2)+')'

for col in range(3, 6, 1):
    sql2.cell(row=index+3, column=col).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')
    
sql2.cell(row=index+6, column=3).value = 'check FOU'
sql2.cell(row=index+6, column=4).value = "=D"+str(index+3)+"-'sql 1'!C" + str(sql1_total)
sql2.cell(row=index+6, column=5).value = "=E"+str(index+3)+"-'sql 1'!D" + str(sql1_total)
    

for col in range(3, 6, 1):
    sql2.cell(row=index+6, column=col).fill = PatternFill(start_color='7994D1', end_color='7994D1', fill_type='solid')

### Select SQL 3

In [57]:
sql3 = file['sql 3']

d3 = pd.read_sql_query("""select counterparty_new_category, sum(ead) ead, sum(rwa) rwa from v_rzb_cdr_full where record_source = 'RETAIL' and counterparty_new_category in ('PI','MICRO') group by counterparty_new_category""", connection_RAY)

### Input the values and add the colors

In [58]:
# add headers
for idx_c, header in enumerate(d3.columns.values):
    sql3.cell(row=1, column=idx_c+1).value = header
    sql3.cell(row=1, column=idx_c+1).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

# Iterate all rows using DataFrame.itertuples()
index = 0
for idx, row in enumerate(d3.itertuples(index = True)):
    for idx_c, header in enumerate(d3.columns.values):
        sql3.cell(row=idx+2, column=idx_c+1).value = getattr(row, header)
        index = idx

# Add the total        
sql3.cell(row=index+3, column=1).value = 'Total:'
sql3.cell(row=index+3, column=2).value = '=SUM(B2:B'+str(index+2)+')'
sql3.cell(row=index+3, column=3).value = '=SUM(C2:C'+str(index+2)+')'

for col in range(1, 4, 1):
    sql3.cell(row=index+3, column=col).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

### Select SQL 4

In [59]:
sql4 = file['sql 4']

d4 = pd.read_sql_query("""select 	
case when table_name = 'FUND' and contract_type ='IS2700'   then' Investment in FUND' else rating_model_new  end as rating_model_new_mapped,	
counterparty_new_type,counterparty_new_type_desc ,counterparty_new_code,counterparty_new_name,rating_model_new,booking_company,risk_country_new ,sum(ead) ead  ,sum(rwa) rwa 	
from rzb_cdr where (rating_model_new='Fund'  or (table_name = 'FUND' and contract_type ='IS2700' ) or (counterparty_new_type='57' and rating_model_new<>'Fund'))	
and  method_new = 'FOU' and NOT (table_name='EQUITY' and sub_method_new='STD')---to remove the grandfathering equities	
group by case when table_name = 'FUND' and contract_type ='IS2700'   then' Investment in FUND' else rating_model_new  end,	
counterparty_new_type,counterparty_new_type_desc ,counterparty_new_code,counterparty_new_name,rating_model_new,booking_company,risk_country_new""", connection_RAY)

### Input the values and add the colors

In [60]:
# add headers
for idx_c, header in enumerate(d4.columns.values):
    sql4.cell(row=1, column=idx_c+1).value = header
    sql4.cell(row=1, column=idx_c+1).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

# Iterate all rows using DataFrame.itertuples()
index = 0
for idx, row in enumerate(d4.itertuples(index = True)):
    for idx_c, header in enumerate(d4.columns.values):
        sql4.cell(row=idx+2, column=idx_c+1).value = getattr(row, header)
        index = idx_c

# Add the total        
sql4.cell(row=1, column=index+4).value = 'ead'
sql4.cell(row=1, column=index+5).value = 'rwa'

sql4.cell(row=2, column=index+3).value = 'check FUND FOU'
sql4.cell(row=2, column=index+4).value = "=SUM(I:I)-SUMIF('sql 1'!B:B,\"Fund\",'sql 1'!C:C)"
sql4.cell(row=2, column=index+5).value = "=SUM(J:J)-SUMIF('sql 1'!B:B,\"Fund\",'sql 1'!D:D)"

for col in range(3, 6, 1):
    sql4.cell(row=2, column=index+col).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

### Select SQL 5

In [61]:
sql5 = file['sql 5']

d5 = pd.read_sql_query("""select sum(ead_pre_ccf) ead_pre_ccf, sum(ead) ead, sum (rwa) rwa, sum(el) el from t_cdr""", connection_RAY)

### Inut the values and add the colors

In [62]:
# add headers
for idx_c, header in enumerate(d5.columns.values):
    sql5.cell(row=1, column=idx_c+1).value = header
    sql5.cell(row=1, column=idx_c+1).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

# Iterate all rows using DataFrame.itertuples()
index = 0
for idx, row in enumerate(d5.itertuples(index = True)):
    for idx_c, header in enumerate(d5.columns.values):
        sql5.cell(row=idx+2, column=idx_c+1).value = getattr(row, header)
        index = idx

        
# Add the total        
sql5.cell(row=index + 4, column=1).value = ' check  total t_Cdr'

sql5.cell(row=index + 5, column=1).value = "=A2-'sql 6'!C11"
sql5.cell(row=index + 5, column=2).value = "=B2-'sql 6'!D11"
sql5.cell(row=index + 5, column=3).value = "=C2-'sql 6'!E11"
sql5.cell(row=index + 5, column=4).value = "=D2-'sql 6'!F11"

for col in range(1, 5, 1):
    sql5.cell(row=index+5, column=col).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

### Select SQL 6

In [63]:
sql6 = file['sql 6']

d6 = pd.read_sql_query("""select securitized, method_new,sum(ead_pre_ccf) ead_pre_ccf, sum(ead) ead, sum (rwa) rwa, sum(el) el 
from t_cdr
group by securitized,method_new""", connection_RAY)

In [64]:
d6 = d6.sort_values(by=["method_new"]) 
d6 = d6[d6.ead > 0]

### Inut the values and add the colors

In [65]:
# add headers
for idx_c, header in enumerate(d6.columns.values):
    sql6.cell(row=1, column=idx_c+1).value = header
    sql6.cell(row=1, column=idx_c+1).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

# Iterate all rows using DataFrame.itertuples()
index = 0
for idx, row in enumerate(d6.itertuples(index = True)):
    for idx_c, header in enumerate(d6.columns.values):
        sql6.cell(row=idx+2, column=idx_c+1).value = getattr(row, header)
        index = idx

        
# Add the total        
sql6.cell(row=index + 3, column=2).value = 'Total'
sql6.cell(row=index + 3, column=3).value = '=SUM(C2:C'+ str(index+2) +')'
sql6.cell(row=index + 3, column=4).value = '=SUM(D2:D'+ str(index+2) +')'
sql6.cell(row=index + 3, column=5).value = '=SUM(E2:E'+ str(index+2) +')'
sql6.cell(row=index + 3, column=6).value = '=SUM(F2:F'+ str(index+2) +')'


for col in range(2, 7, 1):
    sql6.cell(row=index+3, column=col).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

sql6.cell(row=index + 7, column=3).value = "T_CDR"
sql6.cell(row=index + 7, column=4).value = "=SUM(D5,D6)"
sql6.cell(row=index + 7, column=5).value = "=SUM(E5,E6)"

sql6.cell(row=index + 8, column=3).value = "data from rzb_Cdr sql 1"
sql6.cell(row=index + 8, column=4).value = "='sql 1'!C13"
sql6.cell(row=index + 8, column=5).value = "='sql 1'!D13"

sql6.cell(row=index + 9, column=3).value = "Check"
sql6.cell(row=index + 9, column=4).value = "=D15-D16"
sql6.cell(row=index + 9, column=5).value = "=E15-E16"


for col in range(3, 6, 1):
    sql6.cell(row=index+9, column=col).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

### Select SQL 7

In [66]:
sql7 = file['sql 7']

d7 = pd.read_sql_query("""select
case when table_name = 'FUND' and contract_type ='IS2700' then 'Investment in FUND' else rating_model_new end as rating_model_new_mapped
,rating_model_new,
case when (table_name='EQUITY' and sub_method_new='STD') then 'STD' else method_new end as method_new_mapped,
method_new,
sum (ead) ead ,sum (rwa) rwa, sum(el) as el,
count(distinct counterparty_new_code) as number_of_observations, 
max(ead) as max_Ead,
min( ead) as min_ead,
median(ead) as median_ead
from (
select rating_model_new,method_new,sub_method_new,contract_type,table_name,counterparty_new_code,sum (ead) as ead,sum(rwa) as rwa, sum (el_regulatory) as el
from rzb_cdr
where record_source='T_CDR' and ead>0 --as requested by Zsolt
and booking_company not in ('RBBG','RLBG')
group by rating_model_new,method_new,sub_method_new,contract_type,table_name,counterparty_new_code)
group by case when table_name = 'FUND' and contract_type ='IS2700' then 'Investment in FUND' else rating_model_new end, rating_model_new,
case when (table_name='EQUITY' and sub_method_new='STD') then 'STD' else method_new end ,method_new
""", connection_RAY)

In [67]:
# add headers
for idx_c, header in enumerate(d7.columns.values):
    sql7.cell(row=1, column=idx_c+1).value = header
    sql7.cell(row=1, column=idx_c+1).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

# Iterate all rows using DataFrame.itertuples()
index = 0
for idx, row in enumerate(d7.itertuples(index = True)):
    for idx_c, header in enumerate(d7.columns.values):
        sql7.cell(row=idx+2, column=idx_c+1).value = getattr(row, header)
        index = idx

        
# Add the FOU        
sql7.cell(row=index + 11, column=4).value = 'Total FOU'
sql7.cell(row=index + 11, column=5).value = '=SUMIF(C2:C28,"FOU",E2:E28)'
sql7.cell(row=index + 11, column=6).value = '=SUMIF(C2:C28,"FOU",F2:F28)'
sql7.cell(row=index + 11, column=7).value = '=SUMIF(C2:C28,"FOU",G2:G28)'

# Add the STD        
sql7.cell(row=index + 12, column=4).value = 'Total STD'
sql7.cell(row=index + 12, column=5).value = '=SUMIF(C2:C28,"STD",E2:E28)'
sql7.cell(row=index + 12, column=6).value = '=SUMIF(C2:C28,"STD",F2:F28)'
sql7.cell(row=index + 12, column=7).value = '=SUMIF(C2:C28,"STD",G2:G28)'

# Add the ADV        
sql7.cell(row=index + 13, column=4).value = 'Total ADV'
sql7.cell(row=index + 13, column=5).value = '=SUMIF(C2:C28,"ADV",E2:E28)'
sql7.cell(row=index + 13, column=6).value = '=SUMIF(C2:C28,"ADV",F2:F28)'
sql7.cell(row=index + 13, column=7).value = '=SUMIF(C2:C28,"ADV",G2:G28)'

# Add the Total        
sql7.cell(row=index + 14, column=4).value = 'Total'
sql7.cell(row=index + 14, column=5).value = '=SUM(E2:E28)'
sql7.cell(row=index + 14, column=6).value = '=SUM(F2:F28)'
sql7.cell(row=index + 14, column=7).value = '=SUM(G2:G28)'

###
sql7.cell(row=index + 15, column=5).value = '=E38-SUM(E35:E37)'
sql7.cell(row=index + 15, column=6).value = '=F38-SUM(F35:F37)'
sql7.cell(row=index + 15, column=7).value = '=G38-SUM(G35:G37)'

# Add the check fou        
sql7.cell(row=index + 16, column=4).value = 'check FOU'
sql7.cell(row=index + 16, column=5).value = "=E35-'sql 1'!C13"
sql7.cell(row=index + 16, column=6).value = "=F35-'sql 1'!D13"

for col in range(4, 7, 1):
    sql7.cell(row=index+16, column=col).fill = PatternFill(start_color='7994D1', end_color='7994D1', fill_type='solid')

# Add the check total        
sql7.cell(row=index + 17, column=4).value = 'check total'
sql7.cell(row=index + 17, column=5).value = "=E38-SUM('sql 6'!D3;'sql 6'!D6;'sql 6'!D9;'sql 6'!D2;'sql 6'!D5;'sql 6'!D10)"
sql7.cell(row=index + 17, column=6).value = "=F38-SUM('sql 6'!E3;'sql 6'!E6;'sql 6'!E9;'sql 6'!E2;'sql 6'!E5;'sql 6'!E10)"

for col in range(4, 7, 1):
    sql7.cell(row=index+17, column=col).fill = PatternFill(start_color='7994D1', end_color='7994D1', fill_type='solid')

# for col in range(2, 7, 1):
#     sql6.cell(row=index+3, column=col).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

# sql6.cell(row=index + 7, column=3).value = "T_CDR"
# sql6.cell(row=index + 7, column=4).value = "=SUM(D5,D6)"
# sql6.cell(row=index + 7, column=5).value = "=SUM(E5,E6)"

# sql6.cell(row=index + 8, column=3).value = "data from rzb_Cdr sql 1"
# sql6.cell(row=index + 8, column=4).value = "='sql 1'!C13"
# sql6.cell(row=index + 8, column=5).value = "='sql 1'!D13"

# sql6.cell(row=index + 9, column=3).value = "Check"
# sql6.cell(row=index + 9, column=4).value = "=D15-D16"
# sql6.cell(row=index + 9, column=5).value = "=E15-E16"


# for col in range(3, 6, 1):
#     sql6.cell(row=index+9, column=col).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

### Save the file

In [68]:
file.save(file_name)

In [ ]:
framerate = 4410
play_time_seconds = 3

t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
audio_data = np.sin(2*np.pi*300*t) + np.sin(2*np.pi*240*t)
Audio(audio_data, rate=framerate, autoplay=True)